<a href="https://colab.research.google.com/github/Honner/CMWTAT_Digital_Edition/blob/master/docs/sphinx/source/examples/simplified-retrieval-with-colpali-vlm_Vespa-cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<picture>
  <source media="(prefers-color-scheme: dark)" srcset="https://assets.vespa.ai/logos/Vespa-logo-green-RGB.svg">
  <source media="(prefers-color-scheme: light)" srcset="https://assets.vespa.ai/logos/Vespa-logo-dark-RGB.svg">
  <img alt="#Vespa" width="200" src="https://assets.vespa.ai/logos/Vespa-logo-dark-RGB.svg" style="margin-bottom: 25px;">
</picture>

# Scaling ColPALI (VLM) Retrieval

This notebook demonstrates how to represent [ColPali](https://huggingface.co/vidore/colpali) in Vespa
and to scale to large collections. Also see the blog post: [Scaling ColPali to billions of PDFs with Vespa](https://blog.vespa.ai/scaling-colpali-to-billions/)

Consider following the [ColQWen2](https://vespa-engine.github.io/pyvespa/examples/pdf-retrieval-with-ColQwen2-vlm_Vespa-cloud.html) notebook instead as it
use a better model with improved performance (Both accuracy and speed).

ColPali is a powerful visual language model that can generate embeddings for images (screenshots of PDF pages) and text queries.

In this notebook, we will use ColPali to generate embeddings for images of PDF _pages_ and store the embeddings in Vespa.
We will also store the base64 encoded image of the PDF page and meta data like title and url.

We demonstrate how to retrieve relevant pages for a query using the embeddings generated by ColPali.

The TLDR of this notebook:

- Generate an image per PDF page using [pdf2image](https://pypi.org/project/pdf2image/)
and also extract the text using [pypdf](https://pypdf.readthedocs.io/en/stable/user/extract-text.html).
- For each page image, use ColPali to obtain the visual multi-vector embeddings

Then we store visual embeddings in Vespa as a `int8` tensor, where we use a binary compression technique
to reduce the storage footprint by 32x compared to float representations. See [Scaling ColPali to billions of PDFs with Vespa](https://blog.vespa.ai/scaling-colpali-to-billions/)
for details on binarization and using hamming distance for retrieval.

During retrieval time, we use the same ColPali model to generate embeddings for the query and then use Vespa's `nearestNeighbor` query to retrieve the most similar documents
per query vector token, using binary representation with hamming distance. Then we re-rank the results in two phases:

- In the 0-phase we use hamming distance to retrieve the k closest pages per query token vector representation, this is expressed by using multiple nearestNeighbor query operators in Vespa.
- The nearestNeighbor operators exposes pages to the first-phase ranking function, which uses an approximate MaxSim using inverted hamming distance insted of cosine similarity. This is done to reduce the number of pages that are re-ranked in the second phase.
- In the second phase, we perform the full MaxSim operation, using float representations of the embeddings to re-rank the top-k pages from the first phase.

This allows us to scale ColPali to very large collections of PDF pages, while still providing accurate and fast retrieval.

Let us get started.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vespa-engine/pyvespa/blob/master/docs/sphinx/source/examples/simplified-retrieval-with-colpali-vlm_Vespa-cloud.ipynb)


Install dependencies:

Note that the python pdf2image package requires poppler-utils, see other installation options [here](https://pdf2image.readthedocs.io/en/latest/installation.html#installing-poppler).

In [1]:
!sudo apt-get update && sudo apt-get install poppler-utils -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,929 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,520 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-securit

Now install the required python packages:

In [2]:
!pip3 install transformers==4.51.3 accelerate vidore_benchmark==4.0.0 pdf2image pypdf==5.0.1 pyvespa vespacli requests numpy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of colpali-engine to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of colpali-engine to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━

In [1]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from io import BytesIO

from transformers import ColPaliForRetrieval, ColPaliProcessor
from vidore_benchmark.utils.image_utils import scale_image, get_base64_image

### Load the model

This requires that the HF_TOKEN environment variable is set as the underlaying PaliGemma model is hosted on Hugging Face
and has a [restricive licence](https://ai.google.dev/gemma/terms) that requires authentication.

Choose the right device to run the model.

In [2]:
# Load model (bfloat16 support is limited; fallback to float32 if needed)
device = "cuda" if torch.cuda.is_available() else "cpu"
if torch.backends.mps.is_available():
    device = "mps"  # For Apple Silicon devices
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

Load the model and the processor.

In [ ]:
model_name = "vidore/colpali-v1.2-hf"
model = ColPaliForRetrieval.from_pretrained(
    model_name,
    torch_dtype=dtype,
    device_map=device,  # "cpu", "cuda", or "mps" for Apple Silicon
).eval()

processor = ColPaliProcessor.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/862M [00:00<?, ?B/s]

### Working with pdfs

We need to convert a PDF to an array of images. One image per page.
We use the `pdf2image` library for this task. Secondary, we also extract the text contents of the PDF using `pypdf`.

NOTE: This step requires that you have `poppler` installed on your system. Read more in [pdf2image](https://pdf2image.readthedocs.io/en/latest/installation.html) docs.

In [ ]:
import requests
from pdf2image import convert_from_path
from pypdf import PdfReader


def download_pdf(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BytesIO(response.content)
    else:
        raise Exception(f"Failed to download PDF: Status code {response.status_code}")


def get_pdf_images(pdf_url):
    # Download the PDF
    pdf_file = download_pdf(pdf_url)
    # Save the PDF temporarily to disk (pdf2image requires a file path)
    temp_file = "temp.pdf"
    with open(temp_file, "wb") as f:
        f.write(pdf_file.read())
    reader = PdfReader(temp_file)
    page_texts = []
    for page_number in range(len(reader.pages)):
        page = reader.pages[page_number]
        text = page.extract_text()
        page_texts.append(text)
    images = convert_from_path(temp_file)
    assert len(images) == len(page_texts)
    return (images, page_texts)

We define a few sample PDFs to work with. The PDFs are discovered from [this url](https://www.conocophillips.com/company-reports-resources/sustainability-reporting/).

In [ ]:
sample_pdfs = [
    {
        "title": "ConocoPhillips Sustainability Highlights - Nature (24-0976)",
        "url": "https://static.conocophillips.com/files/resources/24-0976-sustainability-highlights_nature.pdf",
    },
    {
        "title": "ConocoPhillips Managing Climate Related Risks",
        "url": "https://static.conocophillips.com/files/resources/conocophillips-2023-managing-climate-related-risks.pdf",
    },
    {
        "title": "ConocoPhillips 2023 Sustainability Report",
        "url": "https://static.conocophillips.com/files/resources/conocophillips-2023-sustainability-report.pdf",
    },
]

Now we can convert the PDFs to images and also extract the text content.

In [ ]:
for pdf in sample_pdfs:
    page_images, page_texts = get_pdf_images(pdf["url"])

    pdf["images"] = page_images
    pdf["texts"] = page_texts

Let us look at the extracted image of the first PDF page. This is the document side input to ColPali, one image per page.

In [ ]:
from IPython.display import display

display(scale_image(sample_pdfs[0]["images"][0], 720))

Let us also look at the extracted text content of the first PDF page.

In [ ]:
print(sample_pdfs[0]["texts"][0])

Notice how the layout and order of the text is different from the image representation. Note that

- The headlines NATURE and Sustainability have been combined into one word (NATURESustainability).
- The 0.03% has been converted to 0.03 and order is not preserved in the text representation.
- The data in the infographics is not represented in the text representation. For example the source water distribution in the infographics is not represented in the text representation.

Now we use the ColPali model to generate embeddings of the images.

In [ ]:
for pdf in sample_pdfs:
    page_embeddings = []
    dataloader = DataLoader(
        pdf["images"],
        batch_size=2,
        shuffle=False,
        collate_fn=lambda x: processor(images=x, return_tensors="pt"),
    )
    for batch_doc in tqdm(dataloader):
        with torch.no_grad():
            batch_doc = {k: v.to(model.device) for k, v in batch_doc.items()}
            embeddings_doc = model(**batch_doc).embeddings
            page_embeddings.extend(list(torch.unbind(embeddings_doc.to("cpu"))))
    pdf["embeddings"] = page_embeddings

Now we are done with the document side embeddings, we now convert the embeddings to Vespa JSON format so we can store (and index) them in Vespa.
Details in [Vespa JSON feed format doc](https://docs.vespa.ai/en/reference/document-json-format.html).


We use binary quantization (BQ) of the page level ColPali vector embeddings to reduce their size by 32x.

Read more about binarization of multi-vector representations in the [colbert blog post](https://blog.vespa.ai/announcing-colbert-embedder-in-vespa/).

The binarization step maps 128 dimensional floats to 128 bits, or 16 bytes per vector. Reducing the size by 32x. On the [DocVQA benchmark](https://huggingface.co/datasets/vidore/docvqa_test_subsampled), binarization results in a small drop in ranking accuracy.

In [ ]:
import numpy as np

vespa_feed = []
for pdf in sample_pdfs:
    url = pdf["url"]
    title = pdf["title"]
    for page_number, (page_text, embedding, image) in enumerate(
        zip(pdf["texts"], pdf["embeddings"], pdf["images"])
    ):
        base_64_image = get_base64_image(scale_image(image, 640), add_url_prefix=False)
        embedding_dict = dict()
        for idx, patch_embedding in enumerate(embedding):
            binary_vector = (
                np.packbits(np.where(patch_embedding > 0, 1, 0))
                .astype(np.int8)
                .tobytes()
                .hex()
            )
            embedding_dict[idx] = binary_vector
        page = {
            "id": hash(url + str(page_number)),
            "fields": {
                "url": url,
                "title": title,
                "page_number": page_number,
                "image": base_64_image,
                "text": page_text,
                "embedding": embedding_dict,
            },
        }
        vespa_feed.append(page)

### Configure Vespa
[PyVespa](https://vespa-engine.github.io/pyvespa/) helps us build the [Vespa application package](https://docs.vespa.ai/en/application-packages.html).
A Vespa application package consists of configuration files, schemas, models, and code (plugins).

First, we define a [Vespa schema](https://docs.vespa.ai/en/schemas.html) with the fields we want to store and their type.

In [ ]:
from vespa.package import Schema, Document, Field, FieldSet, HNSW

colpali_schema = Schema(
    name="pdf_page",
    document=Document(
        fields=[
            Field(
                name="id", type="string", indexing=["summary", "index"], match=["word"]
            ),
            Field(name="url", type="string", indexing=["summary", "index"]),
            Field(
                name="title",
                type="string",
                indexing=["summary", "index"],
                match=["text"],
                index="enable-bm25",
            ),
            Field(name="page_number", type="int", indexing=["summary", "attribute"]),
            Field(name="image", type="raw", indexing=["summary"]),
            Field(
                name="text",
                type="string",
                indexing=["index"],
                match=["text"],
                index="enable-bm25",
            ),
            Field(
                name="embedding",
                type="tensor<int8>(patch{}, v[16])",
                indexing=[
                    "attribute",
                    "index",
                ],  # adds HNSW index for candidate retrieval.
                ann=HNSW(
                    distance_metric="hamming",
                    max_links_per_node=32,
                    neighbors_to_explore_at_insert=400,
                ),
            ),
        ]
    ),
    fieldsets=[FieldSet(name="default", fields=["title", "text"])],
)

Notice the `embedding` field which is a tensor field with the type `tensor<int8>(patch{}, v[16])`.
This is the field we use to represent the patch embeddings from ColPali.

We also enable [HNSW indexing](https://docs.vespa.ai/en/approximate-nn-hnsw.html)
for this field to enable fast nearest neighbor search which is used for candidate retrieval.

We use [binary hamming distance](https://docs.vespa.ai/en/nearest-neighbor-search.html#using-binary-embeddings-with-hamming-distance)
as an approximation of the cosine similarity. Hamming distance is a good approximation
for binary representations, and it is much faster to compute than cosine similarity/dot product.

The `embedding` field is an example of a mixed tensor where we combine one mapped (sparse) dimensions with a dense dimension.

Read more in [Tensor guide](https://docs.vespa.ai/en/tensor-user-guide.html). We also enable [BM25](https://docs.vespa.ai/en/reference/bm25.html) for the `title` and `texts` fields. Notice that the `image` field use type `raw` to store the binary image data, encoded with as a base64 string.

Create the Vespa [application package](https://docs.vespa.ai/en/application-packages):

In [ ]:
from vespa.package import ApplicationPackage

vespa_app_name = "visionrag5"
vespa_application_package = ApplicationPackage(
    name=vespa_app_name, schema=[colpali_schema]
)

Now we define how we want to rank the pages for a query. We use Vespa's support for [BM25](https://docs.vespa.ai/en/reference/bm25.html) for the text, and
late interaction with Max Sim for the image embeddings.

This means that we use the the text representations as a candidate retrieval phase,  then we use the ColPALI embeddings with MaxSim
to rerank the pages.

In [ ]:
from vespa.package import RankProfile, Function, FirstPhaseRanking, SecondPhaseRanking

colpali_profile = RankProfile(
    name="default",
    inputs=[("query(qt)", "tensor<float>(querytoken{}, v[128])")],
    functions=[
        Function(
            name="max_sim",
            expression="""
                sum(
                    reduce(
                        sum(
                            query(qt) * unpack_bits(attribute(embedding)) , v
                        ),
                        max, patch
                    ),
                    querytoken
                )
            """,
        ),
        Function(name="bm25_score", expression="bm25(title) + bm25(text)"),
    ],
    first_phase=FirstPhaseRanking(expression="bm25_score"),
    second_phase=SecondPhaseRanking(expression="max_sim", rerank_count=100),
)
colpali_schema.add_rank_profile(colpali_profile)

The first phase uses a linear combination of BM25 scores for the text fields, and the second phase uses the MaxSim function with the image embeddings. Notice that Vespa supports a `unpack_bits` function to convert the 16 compressed binary vectors to 128-dimensional floats for the MaxSim function. The query input tensor is not compressed and using full float resolution.

### Deploy the application to Vespa Cloud

With the configured application, we can deploy it to [Vespa Cloud](https://cloud.vespa.ai/en/).

To deploy the application to Vespa Cloud we need to create a tenant in the Vespa Cloud:

Create a tenant at [console.vespa-cloud.com](https://console.vespa-cloud.com/) (unless you already have one).
This step requires a Google or GitHub account, and will start your [free trial](https://cloud.vespa.ai/en/free-trial).


In [ ]:
from vespa.deployment import VespaCloud
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Replace with your tenant name from the Vespa Cloud Console
tenant_name = "vespa-team"

key = os.getenv("VESPA_TEAM_API_KEY", None)
if key is not None:
    key = key.replace(r"\n", "\n")  # To parse key correctly

vespa_cloud = VespaCloud(
    tenant=tenant_name,
    application=vespa_app_name,
    key_content=key,  # Key is only used for CI/CD testing of this notebook. Can be removed if logging in interactively
    application_package=vespa_application_package,
)

Now deploy the app to Vespa Cloud dev zone.

The first deployment typically takes 2 minutes until the endpoint is up.


In [ ]:
from vespa.application import Vespa

app: Vespa = vespa_cloud.deploy()

In [ ]:
print("Number of PDF pages:", len(vespa_feed))

Index the documents in Vespa using the Vespa HTTP API.

In [ ]:
from vespa.io import VespaResponse


def callback(response: VespaResponse, id: str):
    if not response.is_successful():
        print(
            f"Failed to feed document {id} with status code {response.status_code}: Reason {response.get_json()}"
        )


# Feed data into Vespa synchronously
app.feed_iterable(vespa_feed, schema="pdf_page", callback=callback)

### Querying Vespa

Ok, so now we have indexed the PDF pages in Vespa. Let us now obtain ColPali embeddings for a few text queries and
use it during ranking of the indexed pdf pages.

Now we can query Vespa with the text query and rerank the results using the ColPali embeddings.


In [ ]:
queries = [
    "Percentage of non-fresh water as source?",
    "Policies related to nature risk?",
    "How much of produced water is recycled?",
]
dataloader = DataLoader(
    queries,
    batch_size=1,
    shuffle=False,
    collate_fn=lambda x: processor(text=x, return_tensors="pt"),
)
qs = []
for batch_query in dataloader:
    with torch.no_grad():
        batch_query = {k: v.to(model.device) for k, v in batch_query.items()}
        embeddings_query = model(**batch_query).embeddings
        qs.extend(list(torch.unbind(embeddings_query.to("cpu"))))

Obtain the query embeddings using the ColPali model:

In [ ]:
dataloader = DataLoader(
    queries,
    batch_size=1,
    shuffle=False,
    collate_fn=lambda x: processor(text=x, return_tensors="pt"),
)
qs = []
for batch_query in dataloader:
    with torch.no_grad():
        batch_query = {k: v.to(model.device) for k, v in batch_query.items()}
        embeddings_query = model(**batch_query).embeddings
        qs.extend(list(torch.unbind(embeddings_query.to("cpu"))))

We create a simple routine to display the results. We render the image and the title of the retrieved page/document.


In [ ]:
from IPython.display import display, HTML


def display_query_results(query, response, hits=5):
    query_time = response.json.get("timing", {}).get("searchtime", -1)
    query_time = round(query_time, 2)
    count = response.json.get("root", {}).get("fields", {}).get("totalCount", 0)
    html_content = f"<h3>Query text: '{query}', query time {query_time}s, count={count}, top results:</h3>"

    for i, hit in enumerate(response.hits[:hits]):
        title = hit["fields"]["title"]
        url = hit["fields"]["url"]
        page = hit["fields"]["page_number"]
        image = hit["fields"]["image"]
        score = hit["relevance"]

        html_content += f"<h4>PDF Result {i + 1}</h4>"
        html_content += f'<p><strong>Title:</strong> <a href="{url}">{title}</a>, page {page+1} with score {score:.2f}</p>'
        html_content += (
            f'<img src="data:image/png;base64,{image}" style="max-width:100%;">'
        )

    display(HTML(html_content))

Query Vespa with the queries and display the results, here we are using the `default` rank profile.

Note that we retrieve using textual representation with `userInput(@userQuery)`, this means that we use the BM25 ranking for the extracted text in the first ranking phase and then re-rank the top-k pages using the ColPali embeddings.

Later in this notebook we will use Vespa's support for approximate nearest neighbor search (`nearestNeighbor`) to retrieve directly using the ColPali embeddings.

In [ ]:
from vespa.io import VespaQueryResponse

async with app.asyncio(connections=1, timeout=120) as session:
    for idx, query in enumerate(queries):
        query_embedding = {k: v.tolist() for k, v in enumerate(qs[idx])}
        response: VespaQueryResponse = await session.query(
            yql="select title,url,image,page_number from pdf_page where userInput(@userQuery)",
            ranking="default",
            userQuery=query,
            timeout=120,
            hits=3,
            body={"input.query(qt)": query_embedding, "presentation.timing": True},
        )
        assert response.is_successful()
        display_query_results(query, response)

### Using nearestNeighbor for retrieval

In the above example, we used the ColPali embeddings in ranking, but using the text query for retrieval.
This is a reasonable approach for text-heavy documents where the text representation is the most important and where ColPali embeddings are used to
re-rank the top-k documents from the text retrieval phase.

In some cases, the ColPali embeddings are the most important and we want
to demonstrate how we can use HNSW indexing with binary hamming distance to retrieve the most similar pages to a query and
then have two steps of re-ranking using the ColPali embeddings.

All the phases here are executed locally inside the Vespa content node(s) so that no vector data needs
to cross the network.

Let us add a new rank-profile to the schema, the `nearestNeighbor` operator takes a query tensor and a field tensor as argument and
we need to define the query tensors types in the rank-profile.

In [ ]:
from vespa.package import RankProfile, Function, FirstPhaseRanking, SecondPhaseRanking

input_query_tensors = []
MAX_QUERY_TERMS = 64
for i in range(MAX_QUERY_TERMS):
    input_query_tensors.append((f"query(rq{i})", "tensor<int8>(v[16])"))

input_query_tensors.append(("query(qt)", "tensor<float>(querytoken{}, v[128])"))
input_query_tensors.append(("query(qtb)", "tensor<int8>(querytoken{}, v[16])"))

colpali_retrieval_profile = RankProfile(
    name="retrieval-and-rerank",
    inputs=input_query_tensors,
    functions=[
        Function(
            name="max_sim",
            expression="""
                sum(
                    reduce(
                        sum(
                            query(qt) * unpack_bits(attribute(embedding)) , v
                        ),
                        max, patch
                    ),
                    querytoken
                )
            """,
        ),
        Function(
            name="max_sim_binary",
            expression="""
                sum(
                  reduce(
                    1/(1 + sum(
                        hamming(query(qtb), attribute(embedding)) ,v)
                    ),
                    max,
                    patch
                  ),
                  querytoken
                )
            """,
        ),
    ],
    first_phase=FirstPhaseRanking(expression="max_sim_binary"),
    second_phase=SecondPhaseRanking(expression="max_sim", rerank_count=10),
)
colpali_schema.add_rank_profile(colpali_retrieval_profile)

We define two functions, one for the first phase and one for the second phase. Instead of the float representations, we use the binary representations with inverted hamming distance in the first phase. Now, we need to re-deploy the application to Vespa Cloud.

In [ ]:
from vespa.application import Vespa

app: Vespa = vespa_cloud.deploy()

Now we can query Vespa with the text queries and use the `nearestNeighbor` operator to retrieve the most similar pages to the query and pass the different query tensors.

In [ ]:
from vespa.io import VespaQueryResponse

target_hits_per_query_tensor = (
    20  # this is a hyper parameter that can be tuned for speed versus accuracy
)
async with app.asyncio(connections=1, timeout=180) as session:
    for idx, query in enumerate(queries):
        float_query_embedding = {k: v.tolist() for k, v in enumerate(qs[idx])}
        binary_query_embeddings = dict()
        for k, v in float_query_embedding.items():
            binary_query_embeddings[k] = (
                np.packbits(np.where(np.array(v) > 0, 1, 0)).astype(np.int8).tolist()
            )

        # The mixed tensors used in MaxSim calculations
        # We use both binary and float representations
        query_tensors = {
            "input.query(qtb)": binary_query_embeddings,
            "input.query(qt)": float_query_embedding,
        }
        # The query tensors used in the nearest neighbor calculations
        for i in range(0, len(binary_query_embeddings)):
            query_tensors[f"input.query(rq{i})"] = binary_query_embeddings[i]
        nn = []
        for i in range(0, len(binary_query_embeddings)):
            nn.append(
                f"({{targetHits:{target_hits_per_query_tensor}}}nearestNeighbor(embedding,rq{i}))"
            )
        # We use a OR operator to combine the nearest neighbor operator
        nn = " OR ".join(nn)
        response: VespaQueryResponse = await session.query(
            yql=f"select title, url, image, page_number from pdf_page where {nn}",
            ranking="retrieval-and-rerank",
            timeout=120,
            hits=3,
            body={**query_tensors, "presentation.timing": True},
        )
        assert response.is_successful()
        display_query_results(query, response)

Depending on the scale, we can evaluate changing different number of targetHits per nearestNeighbor operator and the ranking depths in the two phases. We can also parallelize the ranking phases by using more threads per query request to reduce latency.

## Summary

In this notebook, we have demonstrated how to represent ColPali in Vespa. We have generated embeddings for images of PDF pages using ColPali and stored the embeddings in Vespa using [mixed tensors](https://docs.vespa.ai/en/tensor-user-guide.html).

We demonstrated how to store the base64 encoded image using the `raw` Vespa field type, plus meta data like title and url.
We have demonstrated how to retrieve relevant pages for a query using the embeddings generated by ColPali.

This notebook can be extended to include more complex ranking models, more complex queries, and more complex data structures, including metadata and other fields which can be filtered on or used for ranking.

## Cleanup

When this notebook is running in CI, we want to delete the application.

In [ ]:
if os.getenv("CI", "false") == "true":
    vespa_cloud.delete()